# Read these instructions completely in order to receive full credit

- Before you submit the problem set, make sure everything runs as expected. Go to the menu bar at the top of Jupyter Notebook and click `Kernel > Restart & Run All`. Your code should run from top to bottom with no errors. Failure to do this will result in loss of points.

- You should not use `install.packages()` anywhere. You may assume that we have already installed all the packages needed to run your code.

- Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE" and delete the `stop()` functions. The `stop()` functions produce an error and are there to remind you of cells that need an answer.

- If you are working in a group, make sure you and your collaborators have been added to a group on Canvas as described at the beginning of lecture 2.
- As a backup, *also* fill in your uniqid as well as those of your collaborators below:

Your uniqid: `<replace with your uniqid>`

Uniqids of your collaborator(s): `<replace with their uniqids>`

- **Carefully proofread the PDF that you upload to Canvas. PDFs that have missing or truncated code cannot be graded and will not receive credit.**

---

In [ ]:
library(tidyverse)
library(modelr)
source("rod.R")

# STATS 306
## Problem Set 8
Each problem is worth one point, for a total of 10.

## Part I: Simulation
In Exercises 1-5, you will write code to simulate certain experiments. The solutions to these exercises are examples of the [Monte Carlo method](https://en.wikipedia.org/wiki/Monte_Carlo_method). Numerical answers should be accurate to the nearest hundredth.

### Playing card experiment

Suppose you are given two standard decks of playing cards (52 cards each, total of 104 cards) and you do the following experiment: 
1.  Shuffle both decks thoroughly, so that they are randomly ordered.
2.  Draw the top card off of each deck.
3.  If the two cards exactly match (same suit and rank), award yourself one point; otherwise award yourself no points.
4.  Repeat steps two and three for the remaining 51 pairs of cards.
Let $X$ be your total score at the end of the experiment.

By writing code to simulate this experiment, answer problems 1-3:

#### Problem 1
What the expected value of $X$, i.e. what is $\mathbb{E}X$?

In [ ]:
# YOUR CODE HERE
stop()

#### Problem 2
What is the probability that $X$ is equal to zero? I.e., what is $\mathbb{P}(X=0)$? (*Hint*: the probability that $X=0$ equals the expected value of a random variable which equals one when $X=0$ and zero otherwise.)

In [ ]:
# YOUR CODE HERE
stop()

#### Problem 3
Can you give an exact answer for problem 2? (Hint: consider $1/\mathbb{P}(X=0)$).

In [ ]:
# YOUR CODE HERE
stop()

#### Problem 4
Suppose I repeat the following experiment a large number of times: sample a point $(x,y)$ uniformly at random from $[-1,1]\times[-1,1]$. Let $Z=1$ if the distance from $(x,y)$ to the origin is less than one. What is $\mathbb{E}Z$, the expected value of $Z$? Can you give an exact expression?

In [ ]:
# YOUR CODE HERE
stop()

#### Problem 5
Suppose I repeat the following experiment a large number of times: I draw two parallel lines 1 inch apart on the floor. Then, I drop a 1-inch long rod on the floor so that its center falls somewhere between the two lines.

What is the probability that the rod intersects one of the two lines?

The function `run_rod_experiment()` can be run repeatedly to illustrate how the game is played:

In [ ]:
run_rod_experiment()

In [ ]:
# YOUR CODE HERE
stop()

## Part II: Regression

#### Problem 6
Consider the following regression:

In [ ]:
lm(log(price) ~ log(carat), diamonds) %>% summary

The high $R^2$ and small $p$-values indicate that $\log(\text{price})$ and $\log(\text{carat})$ are approximately linearly related. (We also plotted this relationship in lecture.)

What does this regression say about the relationship between __`price`__ (not `log(price)`) and __`carat`__ (not `log(carat)`)?

YOUR ANSWER HERE

#### Problem 7
Consider the following regression:

In [ ]:
lm(price ~ poly(carat, 2), diamonds) %>% summary

According to this regression, what is the relationship between `price` and `carat`? What effect does increasing `carat` have on price? Compared to the model in problem 6, do you think that this is this a better or worse model for predicting the price of a diamond based on its weight?

#### Problem 8

Pearson's correlation coefficient, denoted $\rho$, measures the strength of a linear relationship between two variables $x$ and $y$. It's implemented using the `cor()` function in R:

In [ ]:
set.seed(1)
x = rnorm(100, sd=.1)
y = 3 + 2 * x + rnorm(100, sd = 1)
cor(x, y)

To test whether a linear relationship exists between two variables, we can use the function `cor.test()` to test the null hypothesis $H_0: \rho=0$:

In [ ]:
cor.test(x, y)

Consider how you might test whether the correlation between $x$ and $y$ is zero using the linear model. By running an appropriate regression, show that you get *the exact same* $p$-value as that obtained by `cor.test()`. How can we infer the correlation coefficient $\rho=0.183$ from the regression result?

In [ ]:
# YOUR CODE HERE
stop()

#### Problem 9
A noted criticism of Pearson's correlation is that it only measures the strength of a *linear* correlation between two random variables. Consider the variables $x$ and $y$ defined in the file `spearman.csv`:

In [ ]:
sp <- read_csv("spearman.csv") %>% print

Are $x$ and $y$ correlated according to Pearson's test? Are they related at all? Support your answer with an appropriate visual or statistical argument.

YOUR ANSWER HERE

In [ ]:
# YOUR CODE HERE
stop()

Recall that the `rank()` function maps a vector to a vector of integers denoting the numerical rank of each entry of the vector:

In [ ]:
rank(c(2,6,9,10,8))

To address potential shortcomings in Pearson's test, *Spearman's rank correlation test* looks for correlations between the *ranks* of two vectors $x$ and $y$. This will do a better job of picking out a non-linear relationship between $x$ and $y$, so long as that relationship is [monotonic](https://en.wikipedia.org/wiki/Monotonic_function). Verify this visualizing the relationship of `rank(x)` and `rank(y)`.

In [ ]:
# YOUR CODE HERE
stop()

Spearman's test is implemented using the `cor.test(..., method="spearman")` command.

Show that regressing the rank of `y` on the rank of `x` gives you you the *exact same* $p$-values and estimates for $\rho$ as Spearman't test.

In [ ]:
# YOUR CODE HERE
stop()

Spearman's test is an example of a [non-parametric](https://en.wikipedia.org/wiki/Nonparametric_statistics) test: it does not make any assumptions about the distribution of the data. We will see other examples of non-parametric tests below.

#### Problem 10
##### The $t$-test
The one-sample $t$-test is used to test the null hypothesis that the mean of a random variable is zero. It's implemented in R using the `t.test()` command:

In [ ]:
x <- rnorm(100, mean=.1)
t.test(x)

By running an appropriate regression, show that the linear model produces *the exact same* $t$ statistic, confidence intervals, and $p$-values as the $t$-test.

In [ ]:
# YOUR CODE HERE
stop()